In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import joblib

# Define custom scorer for MAPE
def mape_scorer(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

# Load the dataset
df_raw = pd.read_csv('../Train.csv')

# Convert the 'date_time' column to datetime and sort the dataset
df_raw['date_time'] = pd.to_datetime(df_raw['date_time'])
df_raw.sort_values('date_time', inplace=True)

# Extracting non-numeric columns
non_numeric_cols = ['is_holiday', 'weather_type', 'weather_description']

# Group by 'date_time' and aggregate: mean for numeric columns, mode for non-numeric columns
agg_funcs = {col: 'mean' for col in df_raw.columns if col not in non_numeric_cols}
agg_funcs.update({col: lambda x: x.mode()[0] if not x.mode().empty else np.nan for col in non_numeric_cols})

df_aggregated = df_raw.groupby('date_time').agg(agg_funcs).reset_index()

# One-hot encode categorical features
df = pd.get_dummies(df_aggregated, columns=non_numeric_cols, drop_first=True)

ValueError: cannot insert date_time, already exists

In [3]:


# Add hour from the 'date_time' column
df['hour'] = df.index.hour

# Remove rows with NaN values resulting from lagged features
df.dropna(inplace=True)

# Split the dataset into features and the target
X = df.drop('traffic_volume', axis=1)
y = df['traffic_volume']

# Split the data into train and test sets
total_samples = df.shape[0]
split_index = int(total_samples * 0.9)

X_train = X.iloc[:split_index]
y_train = y.iloc[:split_index]
X_test = X.iloc[split_index:]
y_test = y.iloc[split_index:]

# Fit the SARIMAX model
model = SARIMAX(y_train, exog=X_train, order=(3, 1, 1), seasonal_order=(1, 1, 1, 12))
results = model.fit(disp=False)

# Save the trained SARIMAX model
joblib.dump(results, 'sarimax_model.joblib')

# Make predictions on the validation set
predictions_sarimax = results.get_forecast(steps=len(X_test), exog=X_test)
predicted_traffic_volume_sarimax = predictions_sarimax.predicted_mean

# Evaluate metrics on the training set for SARIMAX
train_metrics_sarimax = {
    'RMSE': np.sqrt(mean_squared_error(y_train, results.fittedvalues)),
    'MSE': mean_squared_error(y_train, results.fittedvalues),
    'MAE': mean_absolute_error(y_train, results.fittedvalues),
    'MAPE': mape_scorer(y_train, results.fittedvalues)
}

# Evaluate metrics on the validation set for SARIMAX
val_metrics_sarimax = {
    'RMSE': np.sqrt(mean_squared_error(y_test, predicted_traffic_volume_sarimax)),
    'MSE': mean_squared_error(y_test, predicted_traffic_volume_sarimax),
    'MAE': mean_absolute_error(y_test, predicted_traffic_volume_sarimax),
    'MAPE': mape_scorer(y_test, predicted_traffic_volume_sarimax)
}

# Print and compare metrics for SARIMAX
print("Training Set Metrics (SARIMAX):")
print(train_metrics_sarimax)

print("\nValidation Set Metrics (SARIMAX):")
print(val_metrics_sarimax)

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(y_train.index, y_train, label='Train')
plt.plot(y_test.index, y_test, label='Test')
plt.plot(y_test.index, predicted_traffic_volume_sarimax, label='SARIMAX Forecast', linestyle='--')
plt.legend()
plt.title('SARIMAX Forecast vs Actuals')
plt.show()


ValueError: cannot insert date_time, already exists

In [ ]:
# Make predictions on the first 7 days of unseen data
forecast_unseen = results.get_forecast(steps=7*24, exog=X_test.iloc[:7*24])
predicted_traffic_volume_unseen = forecast_unseen.predicted_mean

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(y_test.index[:7*24], y_test[:7*24], label='Actual Unseen')
plt.plot(y_test.index[:7*24], predicted_traffic_volume_unseen, label='Forecast', linestyle='--')
plt.legend()
plt.title('Multivariate SARIMAX Forecast vs Actuals for the First 7 Days of Unseen Data')
plt.xlabel('Date')
plt.ylabel('Traffic Volume')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
